In [ ]:
from utils import r14, r32, r50, r125, r134a, r143a, r170, atom_type, opt_atom_types
import numpy as np
import unyt as u
from fffit.fffit.utils import values_real_to_scaled, values_scaled_to_real, variances_scaled_to_real

In [ ]:
#Load class properies for each molecule
r14_class = r14.R14Constants()
r32_class = r32.R32Constants()
r50_class = r50.R50Constants()
r125_class = r125.R125Constants()
r134a_class = r134a.R134aConstants()
r143a_class = r143a.R143aConstants()
r170_class = r170.R170Constants()

In [ ]:
repeats = 1
seed = 1
save_data = False
at_class = atom_type.AT_Scheme_7()

In [ ]:
molec = "R143a"
molec_data_dict = {molec:r143a_class} #, "R32":r32_class, "R50":r50_class, "R170":r170_class, "R125":r125_class, "R134a":r134a_class, "R143a":r143a_class}
all_gp_dict = opt_atom_types.get_gp_data_from_pkl(list(molec_data_dict.keys()))
# all_gp_dict = {"R14" : {list(all_gp_dict["R14"].keys())[0]: list(all_gp_dict["R14"].values())[0]}}
all_gp_dict = {molec : all_gp_dict[molec]}
setup = opt_atom_types.Problem_Setup(molec_data_dict, all_gp_dict, at_class, save_data)
at_optimizer =  opt_atom_types.Opt_ATs(molec_data_dict, all_gp_dict, at_class, repeats, seed, save_data)
# ls_results = at_optimizer.optimize_ats()
# ls_results

In [ ]:
# Determine the index of the row where column 'B' has the minimum value
min_index = ls_results['Min Obj Cum.'].idxmin()

# Retrieve the row corresponding to the minimum value in column 'B'
min_row = ls_results.iloc[min_index]
print(min_row)

In [ ]:
# ls_results.to_csv("test1.csv")

In [ ]:
molec_data_dict = {"R14":r14_class, "R32":r32_class, "R50":r50_class, "R170":r170_class, "R125":r125_class, "R134a":r134a_class, "R143a":r143a_class}
all_gp_dict = opt_atom_types.get_gp_data_from_pkl(list(molec_data_dict.keys()))
driver = opt_atom_types.Opt_ATs(molec_data_dict, all_gp_dict, at_class, repeats, seed, save_data)